## Common pattern continued

### State Reduction
* The number of states is the product of the number of values of each state variable
  + reducing the states usually reduces the time and space complexity
* Several strategies to reduce the number of states
  + find the relationship between state variables and reduce the number of state variables 
  + reduce the number of state variables from recurrent equation (House robber problem, we remove the rob/nonrob state variable and only used the current house index as the state variable)
  + improve space complexity when the recurrence relation is static (Fibonacci, the current value only depends on the previous two, so we only need to store the previous two values to reduce space complexity)
  
  ```ptyhon
  class Solution:
    def fib(self, n: int) -> int:
        if n <= 1: return n
        one_back = 1
        two_back = 0
        for i in range(2, n + 1):
            temp = one_back
            one_back += two_back
            two_back = temp

        return one_back
   ```

### Leetcode 746 Min Cost Climbing Stairs
* refer to this problem in Strategic Approach to DP notebook

### Counting DP
* A class of DP problems to ask for the number of distinct ways to do something
* difference between counting DP and other DP problems asking for min and max
  + recurrence relationship
    + in DP problems asking for min and max, we usually use the recurrence relationship of max() or min()
    + in counting DP problems, the recurrence relationship typically just sums the results of multiple states together
  + base case
    + in other DP problems, we usually define base cases to return 0
    + in counting DP, we use logic to find the reasonable base case value, for example, in Leetcode 70, Climbing Stairs, we set the base case when i=0 as 1 since there is one way to climb the stair 0

### Leetcode 70 Climbing Stairs
* overview
  + you are climbing a staircase. It takes n steps to reach the top
  + each time, you can climb 1 or 2 steps, how many distince ways can you climb to the top?  

In [1]:
# bottom up with state reduction
class Solution(object):
    def climbStairs(self, n):
        """
        :type n: int
        :rtype: int
        """
        
        pre_one =1
        pre_two =1
        
        for i in range(2,n+1):
            tmp = pre_one
            pre_one+=pre_two
            pre_two =tmp
        return pre_one   